<h1><center>Tagging<center></h1>

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
    
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.
    
    Only extract the properties mentioned in the 'Classification' function.
    Passage:
    {input}
    """
)

class Classification(BaseModel):
    sentiment: str = Field(description = "The sentimet of the text")
    aggresiveness: int = Field(
        description = "How aggresive the text is on a scale from 1 to 10"
    )
    language: str = Field(description = "The language the text is written in")
    

llm = ChatOpenAI(temperature = 0, model = "gpt-4o-mini").with_structured_output(Classification)

In [5]:
input = "Estoy increiblemente contento de haberte concido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": input})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggresiveness=1, language='Spanish')

In [8]:
input_2 = "Man, I really love guinea pigs, they are so cute!"
prompt_2 = tagging_prompt.invoke({"input": input_2})
response_2 = llm.invoke(prompt_2)

response_2.model_dump()

{'sentiment': 'positive', 'aggresiveness': 1, 'language': 'English'}

<h1><center>Finer control<center></h1>

In [9]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum = ["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description = "Describes how aggresive the statement is, the higher then number, the more aggresive",
        enum = [1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum = ["spanish", "english", "french", "german", "italian", "polish", "chinese"]
    )

In [10]:
tagging_prompt_2 = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.
    
    Only extract the properties mentioned in the 'Classification' function.
    
    Passage:
    {input}
    """
)

llm = ChatOpenAI(temperature = 0, model = "gpt-4o-mini").with_structured_output(
    Classification
)

In [12]:
input_3 = "Ah man, I totally forgot about the test and I'm going to fail!"
prompt_3 = tagging_prompt_2.invoke({"input": input_3})
llm.invoke(prompt_3)

Classification(sentiment='sad', aggressiveness=1, language='english')

In [13]:
input_4 = "你好！认识你我很高兴"
prompt_4 = tagging_prompt_2.invoke({"input": input_4})
llm.invoke(prompt_4)

Classification(sentiment='happy', aggressiveness=1, language='chinese')

In [14]:
input_5 = "Wczoraj była dość ładna pogoda, ale niestety dziś cały dzień pada i jest pochmurno"
prompt_5 = tagging_prompt_2.invoke({"input": input_5})
llm.invoke(prompt_5)

Classification(sentiment='neutral', aggressiveness=1, language='polish')